# Balanced Task Assignment with Inverse Cost Scaling

[![Investment_project.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/colab.ampl.com/blob/master/authors/mikhail/Inverse_cost/Inverse_cost.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/colab.ampl.com/blob/master/authors/mikhail/Inverse_cost/Inverse_cost.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/colab.ampl.com/blob/master/authors/mikhail/Inverse_cost/Inverse_cost.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/colab.ampl.com/blob/master/authors/mikhail/Inverse_cost/Inverse_cost.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/colab.ampl.com/blob/master/authors/mikhail/Inverse_cost/Inverse_cost.ipynb) [![Hits](https://h.ampl.com/https://github.com/ampl/colab.ampl.com/blob/master/authors/mikhail/Inverse_cost/Inverse_cost.ipynb)](https://colab.ampl.com)

This model addresses a task assignment problem where workers are assigned to tasks based on a cost matrix. The cost of assigning a task to a worker depends inversely on the number of tasks already assigned to that worker, encouraging balanced task allocation. 
The ***objective*** is to minimize the total cost while ensuring:
- Each task is assigned to exactly one worker.
- Each worker is assigned no more than a specified maximum number of tasks.

[*Partner with the AMPL team to transform complex problems into optimized solutions. AMPL consulting services combine deep technical knowledge with industry-leading insights, helping you unlock the full potential of optimization within your organization.*](https://ampl.com/services/)

Tags: Lessons, Managment, ampl, MINP, 

Notebook author: Mikhail Riabtsev <<mail@solverytic.com>>
***

# 1. Download Necessary Extensions and Libraries

In [1]:
# Install dependencies
%pip install -q amplpy pandas
import pandas as pd                 # Loading panda to work with pandas.DataFrame objects (https://pandas.pydata.org/)
import numpy as np                  # Loading numpy to perform multidimensional calculations numpy.matrix (https://numpy.org/)

Note: you may need to restart the kernel to use updated packages.


In [2]:
'''# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["cbc", "highs", "gurobi"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics'''

'# Google Colab & Kaggle integration\nfrom amplpy import AMPL, ampl_notebook\n\nampl = ampl_notebook(\n    modules=["cbc", "highs", "gurobi"],  # modules to install\n    license_uuid="default",  # license to use\n)  # instantiate AMPL object and register magics'

In [3]:
from amplpy import AMPL
ampl = AMPL()                       # create a new AMPL object with all default settings

# 2. Mathematical Formulation

#### **Sets**
- $(W)$: Set of workers $W = {w_1, w_2, \ldots, w_n}$
- $(T)$: Set of tasks $T = {t_1, t_2, \ldots, t_m}$

#### **Parameters**
- $cost[w, t]$: Cost of assigning worker $w$ to task $t$, where $w \in W $ and $t \in T $.
- $max\_tasks$: Maximum number of tasks that can be assigned to a single worker.

#### **Decision Variables**
- $x[w, t] \in \{0, 1\} $: Binary variable, 1 if worker $w$ is assigned to task $t$, 0 otherwise.

---

#### **Objective Function**

Minimize the total cost of assigning tasks to workers:

$Minimize Z = \sum_{w \in W} \sum_{t \in T} \frac{\text{cost}[w, t]}{1 + \sum_{t' \in T} x[w, t']} \cdot x[w, t]$

---

#### **Constraints**

1. **Task Assignment Constraint**:  
   Each task must be assigned to exactly one worker:

   $ \sum_{w \in W} x[w, t] = 1, \quad \forall t \in T$

2. **Worker Task Limit Constraint**:  
   Each worker can be assigned at most $\text{max\_tasks} $ tasks:

   $\sum_{t \in T} x[w, t] \leq \text{max\_tasks}, \quad \forall w \in W$

3. **Binary Decision Variables**:  
   Ensure the variables are binary:

   $x[w, t] \in \{0, 1\}, \quad \forall w \in W, \, t \in T$

# 3. AMPL Model Formulation

In [4]:
%%writefile Inverse_cost_model.mod
reset;

### SETS
set WORKERS;                        # Set of workers
set TASKS;                          # Set of tasks

### PARAMETERS
param cost {WORKERS, TASKS} >= 0;   # Cost of assigning a worker to a task
param max_tasks integer >= 1;       # Maximum number of tasks per worker

### VARIABLES
var x {WORKERS, TASKS} binary;      # 1 if worker is assigned to task, 0 otherwise

### OBJECTIVE: Minimize the total cost with inverse relationship scaling
# The cost of assigning a worker to a task is divided by 1+ the number of tasks already assigned to that worker. 
# This creates an inverse relationship: the more tasks a worker has, the less "expensive" each additional task becomes.
minimize TotalCost:
    sum {w in WORKERS, t in TASKS} (cost[w, t] / (1 + sum {t2 in TASKS} x[w, t2])) * x[w, t];

### CONSTRAINTS
subject to TaskAssignment {t in TASKS}:
    sum {w in WORKERS} x[w, t] = 1; # Each task must be assigned to exactly one worker

subject to WorkerLimit {w in WORKERS}:
    sum {t in TASKS} x[w, t] <= max_tasks; # Each worker is assigned at most max_tasks tasks

Overwriting Inverse_cost_model.mod


# 4. Load data

In [5]:
ampl.read('Inverse_cost_model.mod')                         # Load the AMPL model from the file
ampl.set['WORKERS'] = ['Alice', 'Bob', 'Carol', 'Dave']     # Set of workers
ampl.set['TASKS'] = ['Task1', 'Task2', 'Task3', 'Task4', 'Task5', 'Task6']  # Set of tasks
ampl.param['max_tasks'] = 3                                 # Maximum number of tasks that can be assigned to a worker                 

ampl.param['cost'] =  {                                     # Cost matrix (cost of assigning a worker to a task)
    ('Alice', 'Task1'): 5, 
    ('Alice', 'Task2'): 8,
    ('Alice', 'Task3'): 6,
    ('Alice', 'Task4'): 7,
    ('Alice', 'Task5'): 5,
    ('Alice', 'Task6'): 8,
    ('Bob', 'Task1'): 7, 
    ('Bob', 'Task2'): 6,
    ('Bob', 'Task3'): 8,
    ('Bob', 'Task4'): 5,
    ('Bob', 'Task5'): 7,
    ('Bob', 'Task6'): 6,
    ('Carol', 'Task1'): 6, 
    ('Carol', 'Task2'): 7,
    ('Carol', 'Task3'): 5,
    ('Carol', 'Task4'): 8,
    ('Carol', 'Task5'): 6,
    ('Carol', 'Task6'): 7,
    ('Dave', 'Task1'): 8, 
    ('Dave', 'Task2'): 5,
    ('Dave', 'Task3'): 7,
    ('Dave', 'Task4'): 6,
    ('Dave', 'Task5'): 8,
    ('Dave', 'Task6'): 5 }

# 5. Solve problem

In [11]:
# Set the solver type for use in solving the problems
solver = 'minos'  # Use CBC solver for optimization tasks

ampl.option['show_stats'] = 0 # Show problem size statistics (default: 0)
ampl.option['display_1col'] = 0 # Disable single-column data display
#ampl.option['omit_zero_rows'] = 1 # Hide rows with zero values
#ampl.option['omit_zero_cols'] = 1 # Hide columns with zero values
ampl.option['mp_options'] = 'outlev=1 lim:time=20'   # Configure CBC options (output level and time limit)

ampl.solve(solver=solver, verbose=False)   # Solve the optimization problem using CBC solver  

# 6. Display results

In [13]:
# Display results for key variables
ampl.display('_varname', '_var', '_var.lb', '_var.ub', '_var.rc', '_var.slack')
ampl.display('_conname', '_con', '_con.body', '_con.lb', '_con.ub', '_con.slack')
ampl.display('_objname', '_obj')

# $4 = _var.ub
:          _varname            _var     _var.lb  $4   _var.rc   _var.slack
 :=
1    "x['Alice','Task1']"   1               0     1   -0.4375   0
2    "x['Alice','Task2']"   0               0     1    0.5625   0
3    "x['Alice','Task3']"   1               0     1    0        0
4    "x['Alice','Task4']"   2.62587e-17     0     1    0        2.62587e-17
5    "x['Alice','Task5']"   1               0     1    0        0
6    "x['Alice','Task6']"   0               0     1    0        0
7    "x['Bob','Task1']"     0               0     1    0        0
8    "x['Bob','Task2']"     1               0     1    0        0
9    "x['Bob','Task3']"     0               0     1    0.4375   0
10   "x['Bob','Task4']"     1               0     1   -0.5625   0
11   "x['Bob','Task5']"     0               0     1    0.4375   0
12   "x['Bob','Task6']"     1               0     1   -0.5625   0
13   "x['Carol','Task1']"   0               0     1    5.3125   0
14   "x['Carol','Task2']"   0         

# 7. Retrieve solution in Python

In [14]:
# Initialize an empty dictionary to store AMPL variable data
amplvar = dict()

# Prepare a list of AMPL variables
list_of_ampl_variables = [item[0] for item in ampl.get_variables()]

# Iterate over each variable name in the list
for key_ampl in list_of_ampl_variables:
    # Skip certain variables that are not to be processed (these variables won't be included in the output)
    if key_ampl not in ['']:
        # Convert the AMPL variable data to a pandas DataFrame
        df = ampl.var[key_ampl].to_pandas()
        # Filter the DataFrame to include only rows where the variable's value is greater than a small threshold (1e-5)
        filtered_df = df[df[f"{key_ampl}.val"] > 1e-5]
        # Round the values in the DataFrame to two decimal places
        rounded_df = filtered_df.round(2)
        # Convert the filtered DataFrame to a dictionary and add it to the amplvar dictionary
        amplvar[key_ampl] = rounded_df #.to_dict(orient='records')
print (amplvar[key_ampl])

               x.val
index0 index1       
Alice  Task1     1.0
       Task3     1.0
       Task5     1.0
Bob    Task2     1.0
       Task4     1.0
       Task6     1.0
